<a href="https://colab.research.google.com/github/m10k1/anthropic-cookbook/blob/main/basic_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-LLM ワークフローの基本

３つのシンプルなマルチLLMワークフローを体験する
これらはコストとレイテンシーをトレードオフにして、タスクのパフォーマンスを向上させる可能性があります

1. プロンプトチェーン (Prompt-Chaining) タスクを一連のサブタスクに分解。前の結果に基づいて次のステップを作成する
2. 並列化 (Parallelization) 独立したサブタスクを複数のLLMに分散し、同時処理を行う。
3. ルーティング(Routing) 入力の特性に基づいて、特化されたLLMパスを動的に選択する。


In [15]:
!pip install anthropic

In [16]:
from google.colab import drive
mount_path = '/content/drive'
drive.mount(mount_path)

import os
import sys
sys.path.append(os.path.join(mount_path,'MyDrive/python'))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
from google.colab import userdata
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

まずはライブラリをインポート

In [20]:
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Callable
from util import llm_call, extract_xml

## プロンプトチェーンパターン

シーケンシャルなプロンプトを順にLLMへ問い合わせる


In [22]:
def chain(input: str, prompts: List[str]) -> str:
    """
    複数のLLMコールを順次連鎖させ、ステップ間で結果を渡す
    """
    result = input
    for i, prompt in enumerate(prompts, 1):
        print(f"\nStep {i}:")
        result = llm_call(f"{prompt}\nInput: {result}")
        print(result)
    return result

## 並列化パターン



In [21]:
def parallel(prompt: str, inputs: List[str], n_workers: int = 3) -> List[str]:
    """
    同じプロンプトを複数のLLMへ同時に処理させる
    """
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}\nInput: {x}") for x in inputs]
        return [f.result() for f in futures]

## ルーティングパターン



In [23]:
def route(input: str, routes: Dict[str, str]) -> str:
    """
    コンテンツ分類を使用して、入力を専用のプロンプトにルーティング
    """
    # First determine appropriate route using LLM with chain-of-thought
    print(f"\n利用可能なルート: {list(routes.keys())}")
    selector_prompt = f"""
    入力を分析して、つぎの選択肢の中から最も適切なルートを選択します: {list(routes.keys())}
    最初に選択した理由を説明し, 選択した結果をXMLフォーマットにして提供してください:

    <reasoning>
    この入力が選択結果にルーティングされるべき理由を簡単に説明してください。
    重要な用語、ユーザの意図、緊急度を考慮してください。
    </reasoning>

    <selection>
    選択したルート名
    </selection>

    Input: {input}""".strip()

    route_response = llm_call(selector_prompt)
    # 応答XMLをパース
    reasoning = extract_xml(route_response, 'reasoning')
    route_key = extract_xml(route_response, 'selection').strip().lower()

    print("Routing Analysis:")
    print(reasoning)
    print(f"\nSelected route: {route_key}")

    # Process input with selected specialized prompt
    selected_prompt = routes[route_key]
    return llm_call(f"{selected_prompt}\nInput: {input}")

## 使用例

それぞれのワークフローを実演する具体的な例

* 構造化データ抽出とフォーマットのためのチェーンワークフロー
* ステークホルダー影響分析のための並列化ワークフロー
* カスタマー・サポート・チケット処理のためのルート・ワークフロー




### 例1：構造化データの抽出とフォーマットのためのチェーンワークフロー
各ステップは生のテキストをフォーマットされた表に徐々に変換する

Step1のプロンプト内容
```
テキストから、数値とそれに関連するメトリクスだけを抽出します。  
それぞれを改行で「value: metric」と書きます。

フォーマット例  
92: 顧客満足度  
45%: 収益成長率  
```
Step2のプロンプト内容
```
可能であれば、すべての数値をパーセンテージに変換する。  
パーセントやポイントでない場合は、小数に変換する（例：92ポイント→92％）

   数字は1行に1つにしてください。  
   フォーマット例  
   92%: 顧客満足度  
   45%：収益の伸び  
```

Step3のプロンプト内容
```
すべての行を数値の降順に並べ替える。  
各行に'value: metric'という書式を維持する。  
   例  
   92％：顧客満足度  
   87％：従業員満足度  
```

ユーザーからの入力
```
第3四半期の業績概要：  
当四半期の顧客満足度スコアは 92 ポイントに上昇。  
売上高は前年比45%増。  
主要市場での市場シェアは現在23％。  
顧客解約率は8%から5%に減少。  
新規ユーザー獲得コストは1ユーザーあたり43ドル。  
製品採用率は78%に上昇。  
従業員満足度は87ポイント。  
営業利益率は34%に改善。  
```


In [24]:
# Example 1: Chain workflow for structured data extraction and formatting
# Each step progressively transforms raw text into a formatted table

data_processing_steps = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",

    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",

    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",

    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |"""
]

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
"""

print("\nInput text:")
print(report)
formatted_result = chain(report, data_processing_steps)
print(formatted_result)


Input text:

Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.


Step 1:
92: customer satisfaction points
45%: revenue growth
23%: market share
5%: customer churn
8%: previous customer churn
$43: user acquisition cost
78%: product adoption rate
87: employee satisfaction points
34%: operating margin

Step 2:
92%: customer satisfaction
45%: revenue growth
23%: market share
5%: customer churn
8%: previous customer churn
43.0: user acquisition cost
78%: product adoption rate
87%: employee satisfaction
34%: operating margin

Step 3:
Here are the lines sorted in descending order by numerical value:

92%: customer satisfaction
87%: employee satisfaction
78%: product adop

### 例2：ステークホルダーの影響分析のための並列化ワークフロー

複数のステークホルダー・グループに対する影響度分析を同時に処理する

顧客、従業員、投資家、サプライヤーの４つステークホルダーに対して分析を行う。

顧客:
- 価格に敏感
- よりよい技術を求めている
- 環境問題に関心がある

従業員:
- 雇用の安定に関心がある
- 新しいスキルを必要としている
- 明確な指示が欲しい

投資家:
- 業務の成長を期待している
- コストの管理を望んでいる
- リスクへの懸念を持っている

サプライヤー:
- 生産能力に制約がある
- 価格圧力がある
- 技術変遷が起きている


市場の変化がこのステークホルダー・グループにどのような影響を与えるかを分析する。  

具体的な影響と推奨行動を提示する。  
明確なセクションと優先順位を持つフォーマット  


In [25]:
# Example 2: Parallelization workflow for stakeholder impact analysis
# Process impact analysis for multiple stakeholder groups concurrently

stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",

    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",

    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",

    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions"""
]

impact_results = parallel(
    """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
    stakeholders
)

for result in impact_results:
    print(result)

MARKET IMPACT ANALYSIS FOR CUSTOMERS

HIGH PRIORITY IMPACTS
--------------------
1. Price Sensitivity
- Rising inflation and costs likely to reduce purchasing power
- Increased competition for value-oriented products
- Risk of trading down to lower-cost alternatives

Recommended Actions:
• Introduce tiered pricing options
• Develop value-focused product lines
• Implement loyalty programs with cost-saving benefits
• Highlight total cost of ownership benefits

2. Technology Demands
- Accelerating tech advancement creating higher expectations
- Integration of AI and smart features becoming standard
- Mobile/digital-first preferences growing

Recommended Actions:
• Accelerate digital transformation initiatives
• Enhance mobile app and online experience
• Add smart features to core products
• Provide tech education and support

MEDIUM PRIORITY IMPACTS
----------------------
3. Environmental Consciousness
- Growing demand for sustainable products
- Increased scrutiny of environmental practic

## 例 3: カスタマー・サポート・チケット処理のためのルート・ワークフロー

 内容分析に基づき、サポートチケットを適切なチームにルーティング



In [ ]:
support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant

    Keep responses professional but friendly.

    Input: """,

    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed

    Use clear, numbered steps and technical details.

    Input: """,

    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time

    Maintain a serious, security-focused tone.

    Input: """,

    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help

    Be educational and encouraging in tone.

    Input: """
}

# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error.
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to
    submit a report by end of day.
    - John""",

    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",

    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    print(f"\nTicket {i}:")
    print("-" * 40)
    print(ticket)
    print("\nResponse:")
    print("-" * 40)
    response = route(ticket, support_routes)
    print(response)